In [3]:
#dowload data
    #https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip 

In [1]:
#import
import os
import zipfile
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline
tf.__version__

'2.0.0'

In [2]:
#โหลดไฟล์แล้วแตกไฟล์ใส่โฟลเดอตามนี้
dataset_path = "C:/tensorflow 2.0/cats_and_dogs_filtered"

In [3]:
train_dir = os.path.join(dataset_path, "train")
validation_dir = os.path.join(dataset_path, "validation")

In [4]:
#สร้างโมเดล
 #โหลด pre-trained model(MobileNetV2)
IMG_SHAPE = (128, 128, 3)
#ตั้งโมเดลกำหนดให้ไร้หัว เพื่อเพิ่มส่วนของเรา(modelของเรา)ใส่ส่วนหัวของมันเพื่อนำมาใช้งานได้
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights="imagenet")

In [5]:
#ดูโครงสร้างของมัน
base_model.summary()

Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
_______________________________________________________________________________

In [6]:
#ทำการ freezing โมเดลเอาไว้ เพือไม่ให้มัน train หรือ เพิ่มค่าเข้าไปใหม่ใน weightเดิมของโมเดล
base_model.trainable = False

In [7]:
#สร้างหัวของเรา(model ที่ใช้ต่อส่วนหัวของ base model)
base_model.output

<tf.Tensor 'out_relu/Identity:0' shape=(None, 4, 4, 1280) dtype=float32>

In [8]:
#ขนาดไม่เหมาะที่จะใช้ สำหรับ หัวของเรา(model ที่ใช้ต่อส่วนหัวของ base model)
#วิธีแก้ไขปันหาในมี 1  เพิ่ม flatten layer เพื่อส่งค่าของ base modelไปที่ส่วนหัวของเรา
#หรือ 2 เพิ่ม global average pooling layer. (คล้ายๆกะ maxpooling layer ใน cnn)
#เราจะใช้ข้อสอง เพราะเราไม่สามารถรับค่าทั้งหมดของ วิธีที่ หนึ่งได้
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
#()คือส่วนที่จะรับinput เข้ามา ในกรณีนี้ คือ base_model.output

In [9]:
global_average_layer

<tf.Tensor 'global_average_pooling2d/Identity:0' shape=(None, 1280) dtype=float32>

In [10]:
#สร้าง prediction_layer สำหรับ classified ดั้งนั้นจึงให้ unit เดียว
prediction_layer = tf.keras.layers.Dense(units=1,activation='sigmoid')(global_average_layer)

In [11]:
#เชือมโมเดล base model กะ หัว(model ที่ใช้ต่อส่วนหัวของ base model)
model = tf.keras.models.Model(inputs=base_model.input, outputs=prediction_layer)

In [12]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
______________________________________________________________________________________________

In [13]:
#เพิ่ม optimizer , loss function , metrics ใส่ในโมเดลของเรา โดยในที่นี้ เราตั้งให้ อัตราการเรียนรู้ อยู่ที่ 0.0001 (สูง)
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),loss="binary_crossentropy",metrics=["accuracy"])

In [14]:
#ต่อเราจะทำการ normalize data (Resizing images)
 #เนืองจาก โครงสร้าง neuron network ใหญ่ๆ แบบนี้จะรับแค่มันซัพพอตเท่านั้น
    #ในกรณีของเรา MobileNet (โครงสร้าง) ซัพพอต: (96, 96), (128, 128), (160, 160), (192, 192), (224, 224).
#เราจะใช้ Data Generators มาช่วยในการ prepocessing data นี้ 
data_gen_train = ImageDataGenerator(rescale=1/255.)#ด้าตาสำหรับ train
data_gen_valid = ImageDataGenerator(rescale=1/255.)#ด้าตาสำหรับ test


In [15]:
#เนื่องจากเราเอาดาต้ามาจาก folder(directory) เราจึงใช้ .flow_from_directory
train_generator = data_gen_train.flow_from_directory(train_dir, target_size=(128,128), batch_size=128, class_mode="binary")

Found 2000 images belonging to 2 classes.


In [16]:
valid_generator = data_gen_valid.flow_from_directory(validation_dir, target_size=(128,128), batch_size=128, class_mode="binary")

Found 1000 images belonging to 2 classes.


In [17]:
#ต่อไปเราจะทำการ train model กันครับ (finally T.T)
model.fit_generator(train_generator, epochs=5, validation_data=valid_generator)

Epoch 1/5


UnknownError: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above. [Op:Conv2D]

In [18]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [19]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)
print("Accuracy after transfer learning: {}".format(valid_accuracy))

UnknownError: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above. [Op:Conv2D]

In [20]:
#fine tuning
base_model.trainable = True

In [21]:
print("Number of layersin the base model: {}".format(len(base_model.layers)))

Number of layersin the base model: 155


In [22]:
fine_tune_at = 100

In [23]:
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

In [24]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [25]:
model.fit_generator(train_generator,  
                    epochs=5, 
                    validation_data=valid_generator)

Epoch 1/5


UnknownError: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above. [Op:Conv2D]

In [ ]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)
print("Validation accuracy after fine tuning: {}".format(valid_accuracy))